In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/validdata/sample_valid_submission (1).csv
/kaggle/input/valid-result/valid_result_R.csv
/kaggle/input/valid-org/valid_org.csv
/kaggle/input/bigcontest/2021 ___Biz_.xlsx
/kaggle/input/bigcontest/2021 ___Biz__update_210831.xlsx
/kaggle/input/train-data/2021 ___Biz_.csv


# Data Load

In [6]:
train = pd.read_csv("../input/train-data/2021 ___Biz_.csv",encoding='CP949')
train

,REG_DATE,P_TYPE,CTRY_1,CTRY_2,P_PURPOSE,CATEGORY_1,CATEGORY_2,P_NAME,P_IMPORT_TYPE,P_PRICE
0,2015-12-28,수산물,아르헨티나,아르헨티나,판매용,갑각류,새우,아르헨티나붉은새우,냉동,7.48
1,2015-12-28,수산물,바레인,바레인,판매용,갑각류,게,꽃게,냉동,2.92
2,2015-12-28,수산물,바레인,바레인,판매용,갑각류,게,꽃게,"냉동,절단",3.36
3,2015-12-28,수산물,칠레,칠레,판매용,패류 멍게류,해삼,해삼,"건조,자숙",18.26
4,2015-12-28,수산물,중국,중국,판매용,어류,서대 박대 페루다,서대,냉동,4.79
...,...,...,...,...,...,...,...,...,...,...
42068,2019-12-30,수산물,러시아,러시아,판매용,갑각류,게,왕게,활,38.44
42069,2019-12-30,수산물,중국,중국,판매용,연체류 해물모듬,낙지,낙지,활,8.28
42070,2019-12-30,수산물,러시아,러시아,판매용,갑각류,게,대게,활,23.40
42071,2019-12-30,수산물,중국,중국,판매용,패류 멍게류,바지락,바지락,활,0.85


In [7]:
len(train.P_IMPORT_TYPE.unique())

78

# 타입재정리

In [8]:
def convert_target_datatype(x):
    return float(x.replace(',', '').rstrip())

In [9]:
train.P_PRICE = train.P_PRICE.apply(convert_target_datatype)

In [10]:
def convert_target_datatype_type(x):
    return (x.split(','))

In [11]:
train.P_IMPORT_TYPE = train.P_IMPORT_TYPE.apply(convert_target_datatype_type)

In [12]:
train['storage'] = None
train['part'] = None
train['process'] = None
train['sashimi'] = None
train['steaming_processing'] = None

In [13]:
type_1 = ['건조','냉동','냉장','염장','활']
type_2 = ['간','개아지살','곤이','껍질','꼬리_외화획득용','난포선','내장','눈살','다리','동체','머리','머리_외화획득용','머리살','목살','볼살','알','외투막','지느러미','집게다리','창난','턱살','줄기','살']
type_3 = ['절단','슬라이스(S)','필렛(F)','한쪽껍질붙은','훈제','캐비아대용']
type_4 = ['포장횟감','횟감']
type_5 = ['자숙']

In [14]:
for i in range(len(train)):
    x = train.P_IMPORT_TYPE[i]
    if len(x) == 1: 
        if x[0] in type_1:
            train.storage[i] = x[0]
        elif x[0] in type_2:
            train.part[i] = x[0]
        elif x[0] in type_3:
            train.process[i] = x[0]
        elif x[0] in type_4:
            train.sashimi[i] = x[0]            
        elif x[0] in type_5:
            train.steaming_processing.append[i] = x[0]
    elif len(x) >= 2:
        for j in range(len(x)):
            if x[j] in type_1:
                train.storage[i] = x[j]
            elif x[j] in type_2:
                train.part[i] = x[j]
            elif x[j] in type_3:
                train.process[i] = x[j]
            elif x[j] in type_4:
                train.sashimi[i] = x[j]            
            elif x[j] in type_5:
                train.steaming_processing[i] = x[j]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying

# REG_DATE 전처리
- REG_DATE를 year, month, day로 나눠서 각 컬럼을 추가한다
- 각 월의 day를 기준으로 1~5주차로 나눠 컬럼을 추가한다

In [15]:
train["YEAR"] = train.REG_DATE.apply(lambda x: x.split('-')[0])
train["MONTH"] = train.REG_DATE.apply(lambda x: x.split('-')[1])
train["DAY"] = train.REG_DATE.apply(lambda x: x.split('-')[2])

In [16]:
train['REG_DATE'] = pd.to_datetime(train['REG_DATE'], format='%Y/%m/%d')

In [17]:
train['주차'] = None

In [18]:
_2015 = train[train.REG_DATE.dt.year == 2015]

#### 2015년은 12월 28일 한 주 밖에 없어 2016년 1월 첫주차로 생각 

In [19]:
_2015['주차'] = 1

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [20]:
_2016 = train[train.REG_DATE.dt.year == 2016]
_2017 = train[train.REG_DATE.dt.year == 2017]
_2018 = train[train.REG_DATE.dt.year == 2018]
_2019 = train[train.REG_DATE.dt.year == 2019]

In [21]:
max_ = 32
w = 2
for i in _2016.index:
     if _2016.REG_DATE[i].month == 1:
            x = _2016.REG_DATE[i].day
            if max_ >= x:
                max_ = x
                _2016['주차'][i] = w
            else:
                max_ = x
                w += 1
                _2016['주차'][i] = w

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


#### 2015년 12월 28일을 2016년 1월 첫주차로 생각했기 때문에 2016년 1월 데이터는 2주차 부터 시작 

In [22]:
for j in range(2,13):
    max_ = 32
    w = 1
    for i in _2016.index:
        if _2016.REG_DATE[i].month == j:
            x = _2016.REG_DATE[i].day
            if max_ >= x:
                max_ = x
                _2016['주차'][i] = w
            else:
                max_ = x
                w += 1
                _2016['주차'][i] = w

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [23]:
for j in range(1,13):
    max_ = 32
    w = 1
    for i in _2017.index:
        if _2017.REG_DATE[i].month == j:
            x = _2017.REG_DATE[i].day
            if max_ >= x:
                max_ = x
                _2017['주차'][i] = w
            else:
                max_ = x
                w += 1
                _2017['주차'][i] = w

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [24]:
for j in range(1,13):
    max_ = 32
    w = 1
    for i in _2018.index:
        if _2018.REG_DATE[i].month == j:
            x = _2018.REG_DATE[i].day
            if max_ >= x:
                max_ = x
                _2018['주차'][i] = w
            else:
                max_ = x
                w += 1
                _2018['주차'][i] = w

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [25]:
for j in range(1,13):
    max_ = 32
    w = 1
    for i in _2019.index:
        if _2019.REG_DATE[i].month == j:
            x = _2019.REG_DATE[i].day
            if max_ >= x:
                max_ = x
                _2019['주차'][i] = w
            else:
                max_ = x
                w += 1
                _2019['주차'][i] = w

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [26]:
train = pd.concat([_2015,_2016,_2017,_2018,_2019])

In [27]:
_tmp = train.drop_duplicates(['REG_DATE'])
_tmp[['REG_DATE','YEAR','MONTH','DAY','주차']]

,REG_DATE,YEAR,MONTH,DAY,주차
0,2015-12-28,2015,12,28,1
199,2016-01-04,2016,01,04,2
414,2016-01-11,2016,01,11,3
635,2016-01-18,2016,01,18,4
824,2016-01-25,2016,01,25,5
...,...,...,...,...,...
41025,2019-12-02,2019,12,02,1
41234,2019-12-09,2019,12,09,2
41446,2019-12-16,2019,12,16,3
41672,2019-12-23,2019,12,23,4


# Train

### - 라벨인코딩

In [28]:
from sklearn.preprocessing import LabelEncoder

In [29]:
train.columns

Index(['REG_DATE', 'P_TYPE', 'CTRY_1', 'CTRY_2', 'P_PURPOSE', 'CATEGORY_1',
       'CATEGORY_2', 'P_NAME', 'P_IMPORT_TYPE', 'P_PRICE', 'storage', 'part',
       'process', 'sashimi', 'steaming_processing', 'YEAR', 'MONTH', 'DAY',
       '주차'],
      dtype='object')

In [30]:
str_FEATS = ['CTRY_1', 'CTRY_2', 'P_PURPOSE','P_NAME','storage', 'part',
       'process', 'sashimi', 'steaming_processing']

In [31]:
train = train.fillna('null')

In [32]:
for feat in str_FEATS:
    encoder = LabelEncoder()
    train[feat+'_le'] = encoder.fit_transform(train[feat])

In [33]:
train.columns

Index(['REG_DATE', 'P_TYPE', 'CTRY_1', 'CTRY_2', 'P_PURPOSE', 'CATEGORY_1',
       'CATEGORY_2', 'P_NAME', 'P_IMPORT_TYPE', 'P_PRICE', 'storage', 'part',
       'process', 'sashimi', 'steaming_processing', 'YEAR', 'MONTH', 'DAY',
       '주차', 'CTRY_1_le', 'CTRY_2_le', 'P_PURPOSE_le', 'P_NAME_le',
       'storage_le', 'part_le', 'process_le', 'sashimi_le',
       'steaming_processing_le'],
      dtype='object')

In [34]:
train[['CTRY_1', 'CTRY_2', 'P_PURPOSE','P_NAME','storage', 'part',
       'process', 'sashimi', 'steaming_processing']]

,CTRY_1,CTRY_2,P_PURPOSE,P_NAME,storage,part,process,sashimi,steaming_processing
0,아르헨티나,아르헨티나,판매용,아르헨티나붉은새우,냉동,null,null,null,null
1,바레인,바레인,판매용,꽃게,냉동,null,null,null,null
2,바레인,바레인,판매용,꽃게,냉동,null,절단,null,null
3,칠레,칠레,판매용,해삼,건조,null,null,null,자숙
4,중국,중국,판매용,서대,냉동,null,null,null,null
...,...,...,...,...,...,...,...,...,...
42068,러시아,러시아,판매용,왕게,활,null,null,null,null
42069,중국,중국,판매용,낙지,활,null,null,null,null
42070,러시아,러시아,판매용,대게,활,null,null,null,null
42071,중국,중국,판매용,바지락,활,null,null,null,null


In [35]:
train[['CTRY_1_le', 'CTRY_2_le', 'P_PURPOSE_le', 'P_NAME_le',
       'storage_le', 'part_le', 'process_le', 'sashimi_le',
       'steaming_processing_le']]

,CTRY_1_le,CTRY_2_le,P_PURPOSE_le,P_NAME_le,storage_le,part_le,process_le,sashimi_le,steaming_processing_le
0,48,49,3,172,1,0,0,0,0
1,30,29,3,48,1,0,0,0,0
2,30,29,3,48,1,0,2,0,0
3,66,66,3,265,0,0,0,0,1
4,65,64,3,158,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...
42068,16,15,3,187,4,0,0,0,0
42069,65,64,3,49,4,0,0,0,0
42070,16,15,3,64,4,0,0,0,0
42071,65,64,3,120,4,0,0,0,0


In [36]:
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

In [37]:
params = {
    "iterations": 500,
    "learning_rate": 0.1,
    "task_type": "GPU",
    "devices": "0:1",
    'one_hot_max_size':256,
    'scale_pos_weight':1,
    'objective':'multi:softmax',
    'random_state':2
    
}

In [38]:
model = XGBClassifier(**params)

# Valid

#### 제공된 자율평가데이터로부터 사용하기 쉽게 형태를 바꿨다.

In [39]:
valid = pd.read_csv('../input/validdata/sample_valid_submission (1).csv',encoding='CP949')
valid = valid.sort_values(by='REG_DATE')

In [40]:
from copy import deepcopy

In [41]:
!pip install openpyxl

     |████████████████████████████████| 244 kB 893 kB/s eta 0:00:01


In [42]:
data = pd.read_excel('../input/bigcontest/2021 ___Biz_.xlsx')

In [43]:
make_valid = pd.DataFrame(columns=['REG_DATE', 'CATEGORY_2', 'P_NAME', 'P_PRICE'])

In [44]:
make_valid['REG_DATE'] = data['REG_DATE'].unique()
make_valid['P_PRICE'] = 0
squid = deepcopy(make_valid)
salmon = deepcopy(make_valid)
shrimp = deepcopy(make_valid)

In [45]:
squid['CATEGORY_2'] = '오징어'
squid['P_NAME'] = '오징어'
salmon['CATEGORY_2'] = '연어'
salmon['P_NAME'] = '연어'
shrimp['CATEGORY_2'] = '새우'
shrimp['P_NAME'] = '흰다리새우'

In [46]:
valid = pd.concat([squid, salmon, shrimp]).reset_index(drop=True)

In [47]:
valid['YEAR'] = valid['REG_DATE'].dt.year
valid['MONTH'] = valid['REG_DATE'].dt.month
valid['DAY'] = valid['REG_DATE'].dt.day

In [48]:
valid['주차'] = None

In [49]:
for j in range(1,13):
    max_ = 32
    w = 1
    for i in valid.index:
        if valid.REG_DATE[i].month == j:
            x = valid.REG_DATE[i].day
            if max_ >= x:
                max_ = x
                valid['주차'][i] = w
            else:
                max_ = x
                w += 1
                valid['주차'][i] = w

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


# CTRY_1 & CTRY_2 & P_PURPOSE 

In [50]:
tmp = pd.DataFrame()
j = 0

for i in range(len(valid)):
    cond1 = train['P_NAME']==valid.loc[i, 'P_NAME']
    cond2 = train['MONTH']==valid.loc[i, 'MONTH']
    cond3 = train['주차']==valid.loc[i, '주차']
    cond = False
    for a, b, c in train.loc[cond1 & cond2 & cond3, ['CTRY_1', 'CTRY_2', 'P_PURPOSE']].value_counts().sort_values(ascending=False)[:3].index:
        cond = True
        tmp.loc[j, 'REG_DATE'] = valid.loc[i, 'REG_DATE']
        tmp.loc[j, 'P_NAME'] = valid.loc[i, 'P_NAME']
        tmp.loc[j, 'CTRY_1'] = a
        tmp.loc[j, 'CTRY_2'] = b
        tmp.loc[j, 'P_PURPOSE'] = c
        j += 1
    if not cond:
        for a, b, c in train.loc[cond1, ['CTRY_1', 'CTRY_2','P_PURPOSE']].value_counts()[:3].index:
            tmp.loc[j, 'REG_DATE'] = valid.loc[i, 'REG_DATE']
            tmp.loc[j, 'P_NAME'] = valid.loc[i, 'P_NAME']
            tmp.loc[j, 'CTRY_1'] = a
            tmp.loc[j, 'CTRY_2'] = b
            tmp.loc[j, 'P_PURPOSE'] = c
            j += 1

In [51]:
valid = tmp.merge(valid, how='outer', on=['REG_DATE', 'P_NAME'])

# STORAGE 예측

In [52]:
valid = valid.fillna('null')

In [53]:
train['주차'] = train['주차'].astype(int)
train['YEAR'] = train['YEAR'].astype(int)
train['MONTH'] = train['MONTH'].astype(int)

In [54]:
valid['주차'] = valid['주차'].astype(int)
valid['YEAR'] = valid['YEAR'].astype(int)
valid['MONTH'] = valid['MONTH'].astype(int)

In [55]:
str_FEATS = ['CTRY_1', 'CTRY_2', 'P_NAME', 'P_PURPOSE',]

In [56]:
for feat in str_FEATS:
    valid[feat+'_le'] = valid[feat].apply(lambda x: train.loc[train[feat]==x, feat+'_le'].unique()[0])

In [57]:
FEATS = ['주차', 'YEAR', 'MONTH','CTRY_1_le', 'CTRY_2_le',
       'P_NAME_le','P_PURPOSE_le']
TARGET = ['storage_le']

In [58]:
model = XGBClassifier(**params)

In [59]:
model.fit(train[FEATS], train[TARGET])

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


[04:10:16] WARNING: ../src/learner.cc:573: 
Parameters: { "devices", "iterations", "one_hot_max_size", "scale_pos_weight", "task_type" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[04:10:16] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, devices='0:1', gamma=0,
              gpu_id=-1, importance_type='gain', interaction_constraints='',
              iterations=500, learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=2, num_parallel_tree=1,
              objective='multi:softprob', one_hot_max_size=256, random_state=2,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              task_type='GPU', tree_method='exact', validate_parameters=1, ...)

In [60]:
predition = model.predict(valid[FEATS])
valid['storage_le'] = predition

In [61]:
valid.storage_le.value_counts()

1    312
2    156
Name: storage_le, dtype: int64

# PART 예측

In [62]:
FEATS = [ 'YEAR', 'MONTH', '주차', 'CTRY_1_le', 'CTRY_2_le', 'P_NAME_le','storage_le', 'P_PURPOSE_le']
TARGET = ['part_le']

In [63]:
model = XGBClassifier(**params)

In [64]:
model.fit(train[FEATS], train[TARGET])

[04:10:24] WARNING: ../src/learner.cc:573: 
Parameters: { "devices", "iterations", "one_hot_max_size", "scale_pos_weight", "task_type" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[04:10:24] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, devices='0:1', gamma=0,
              gpu_id=-1, importance_type='gain', interaction_constraints='',
              iterations=500, learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=2, num_parallel_tree=1,
              objective='multi:softprob', one_hot_max_size=256, random_state=2,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              task_type='GPU', tree_method='exact', validate_parameters=1, ...)

In [65]:
predition = model.predict(valid[FEATS])
valid['part_le'] = predition

# PROCESS 예측

In [66]:
FEATS = [ 'YEAR', 'MONTH', '주차', 'CTRY_1_le', 'CTRY_2_le', 'P_NAME_le','storage_le','part_le', 'P_PURPOSE_le']
TARGET = ['process_le']

In [67]:
model = XGBClassifier(**params)

In [68]:
model.fit(train[FEATS], train[TARGET])

[04:10:50] WARNING: ../src/learner.cc:573: 
Parameters: { "devices", "iterations", "one_hot_max_size", "scale_pos_weight", "task_type" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[04:10:50] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, devices='0:1', gamma=0,
              gpu_id=-1, importance_type='gain', interaction_constraints='',
              iterations=500, learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=2, num_parallel_tree=1,
              objective='multi:softprob', one_hot_max_size=256, random_state=2,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              task_type='GPU', tree_method='exact', validate_parameters=1, ...)

In [69]:
predition = model.predict(valid[FEATS])
valid['process_le'] = predition

# SASHIMI 예측

In [70]:
FEATS = [ 'YEAR', 'MONTH', '주차', 'CTRY_1_le', 'CTRY_2_le', 'P_NAME_le','storage_le','part_le', 'P_PURPOSE_le']
TARGET = ['sashimi_le']

In [71]:
model = XGBClassifier(**params)

In [72]:
model.fit(train[FEATS], train[TARGET])

[04:10:59] WARNING: ../src/learner.cc:573: 
Parameters: { "devices", "iterations", "one_hot_max_size", "scale_pos_weight", "task_type" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[04:10:59] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, devices='0:1', gamma=0,
              gpu_id=-1, importance_type='gain', interaction_constraints='',
              iterations=500, learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=2, num_parallel_tree=1,
              objective='multi:softprob', one_hot_max_size=256, random_state=2,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              task_type='GPU', tree_method='exact', validate_parameters=1, ...)

In [73]:
predition = model.predict(valid[FEATS])
valid['sashimi_le'] = predition

# STEAMING_PROCESSING 예측

In [135]:
params = {
    "iterations": 500,
    "learning_rate": 0.1,
    "task_type": "GPU",
    "devices": "0:1",
    'one_hot_max_size':256,
    'random_state':2
}

In [136]:
model_C = CatBoostClassifier(**params)

In [137]:
FEATS = [ 'YEAR', 'MONTH', '주차', 'CTRY_1_le', 'CTRY_2_le', 'P_NAME_le','storage_le','part_le','sashimi_le', 'P_PURPOSE_le']
TARGET = ['steaming_processing_le']

In [138]:
model_C.fit(train[FEATS], train[TARGET])

0:	learn: 0.4939278	total: 70.9ms	remaining: 35.4s
1:	learn: 0.3746447	total: 136ms	remaining: 33.9s
2:	learn: 0.3075324	total: 206ms	remaining: 34.1s
3:	learn: 0.2631550	total: 269ms	remaining: 33.3s
4:	learn: 0.2321036	total: 334ms	remaining: 33s
5:	learn: 0.2096915	total: 397ms	remaining: 32.6s
6:	learn: 0.1935464	total: 460ms	remaining: 32.4s
7:	learn: 0.1820895	total: 522ms	remaining: 32.1s
8:	learn: 0.1738029	total: 587ms	remaining: 32s
9:	learn: 0.1671025	total: 649ms	remaining: 31.8s
10:	learn: 0.1609670	total: 712ms	remaining: 31.6s
11:	learn: 0.1559244	total: 775ms	remaining: 31.5s
12:	learn: 0.1515219	total: 840ms	remaining: 31.5s
13:	learn: 0.1480274	total: 903ms	remaining: 31.3s
14:	learn: 0.1438441	total: 965ms	remaining: 31.2s
15:	learn: 0.1385941	total: 1.03s	remaining: 31.1s
16:	learn: 0.1358837	total: 1.09s	remaining: 31.1s
17:	learn: 0.1330081	total: 1.16s	remaining: 31s
18:	learn: 0.1291795	total: 1.22s	remaining: 30.9s
19:	learn: 0.1273272	total: 1.28s	remaining: 3

In [139]:
predition = model_C.predict(valid[FEATS])
valid['steaming_processing_le'] = predition

# P_PRICE 예측

In [140]:
from xgboost import XGBRegressor

In [141]:
params = {
    "iterations": 500,
    "learning_rate": 0.1,
    "task_type": "GPU",
    "devices": "0:1",
    'one_hot_max_size':256,
    'objective':'reg:linear',
    'random_state':2,
    'scale_pos_weight':1
}


In [142]:
FEATS = ['주차','MONTH','YEAR','CTRY_1_le','CTRY_2_le', 'P_NAME_le', 'storage_le', 'part_le',
       'process_le', 'sashimi_le', 'steaming_processing_le', 'P_PURPOSE_le']
TARGETS = ['P_PRICE']

In [143]:
model = XGBRegressor(**params)

In [144]:
model.fit(train[FEATS], train[TARGETS])

[04:15:24] WARNING: ../src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[04:15:24] WARNING: ../src/learner.cc:573: 
Parameters: { "devices", "iterations", "one_hot_max_size", "task_type" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, devices='0:1', gamma=0,
             gpu_id=-1, importance_type='gain', interaction_constraints='',
             iterations=500, learning_rate=0.1, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=2, num_parallel_tree=1,
             objective='reg:linear', one_hot_max_size=256, random_state=2,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             task_type='GPU', tree_method='exact', validate_parameters=1, ...)

In [145]:
pred = model.predict(valid[FEATS])
valid["P_PRICE"] = pred

# 예측값 VS 실제값 비교 

In [146]:
valid_gt = pd.read_csv("../input/valid-org/valid_org.csv", encoding="CP949")

In [147]:
pred1 = valid.loc[valid['P_NAME']=="오징어", :].groupby(["REG_DATE"])["P_PRICE"].mean().tolist()
pred2 = valid.loc[valid['P_NAME']=="연어", :].groupby(["REG_DATE"])["P_PRICE"].mean().tolist()
pred3 = valid.loc[valid['P_NAME']=="흰다리새우", :].groupby(["REG_DATE"])["P_PRICE"].mean().tolist()

prediction = pred1 + pred2 + pred3

In [148]:
gt1 = valid_gt.loc[valid_gt['P_NAME']=="오징어", :].groupby(["REG_DATE"])["P_PRICE"].mean().tolist()
gt2 = valid_gt.loc[valid_gt['P_NAME']=="연어", :].groupby(["REG_DATE"])["P_PRICE"].mean().tolist()
gt3 = valid_gt.loc[valid_gt['P_NAME']=="흰다리새우", :].groupby(["REG_DATE"])["P_PRICE"].mean().tolist()

ground_truth = gt1 + gt2 + gt3

In [149]:
from sklearn.metrics import mean_squared_error

def rmse(y, y_pred):
    return mean_squared_error(y, y_pred)**0.5
rmse(ground_truth, prediction)

1.2298850191122614

In [150]:
for a, b in zip(FEATS, model.feature_importances_):
    print(a, b)

주차 0.05021935
MONTH 0.011416837
YEAR 0.0050181183
CTRY_1_le 0.07469623
CTRY_2_le 0.017206483
P_NAME_le 0.015486821
storage_le 0.12454991
part_le 0.006913515
process_le 0.23939598
sashimi_le 0.32472345
steaming_processing_le 0.12651905
P_PURPOSE_le 0.0038542484


### 날짜 data 제거 & P_PURPOSE 제거

In [151]:
FEATS = ['CTRY_1_le','CTRY_2_le', 'P_NAME_le', 'storage_le', 'part_le',
       'process_le', 'sashimi_le', 'steaming_processing_le']
TARGETS = ['P_PRICE']

In [152]:
model = XGBRegressor(**params)

In [153]:
model.fit(train[FEATS], train[TARGETS])

[04:15:27] WARNING: ../src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[04:15:27] WARNING: ../src/learner.cc:573: 
Parameters: { "devices", "iterations", "one_hot_max_size", "task_type" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, devices='0:1', gamma=0,
             gpu_id=-1, importance_type='gain', interaction_constraints='',
             iterations=500, learning_rate=0.1, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=2, num_parallel_tree=1,
             objective='reg:linear', one_hot_max_size=256, random_state=2,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             task_type='GPU', tree_method='exact', validate_parameters=1, ...)

In [154]:
pred = model.predict(valid[FEATS])
valid["P_PRICE"] = pred

In [155]:
pred1 = valid.loc[valid['P_NAME']=="오징어", :].groupby(["REG_DATE"])["P_PRICE"].mean().tolist()
pred2 = valid.loc[valid['P_NAME']=="연어", :].groupby(["REG_DATE"])["P_PRICE"].mean().tolist()
pred3 = valid.loc[valid['P_NAME']=="흰다리새우", :].groupby(["REG_DATE"])["P_PRICE"].mean().tolist()

prediction = pred1 + pred2 + pred3

In [156]:
rmse(ground_truth, prediction)

1.0651528904195824

In [157]:
for a, b in zip(FEATS, model.feature_importances_):
    print(a, b)

CTRY_1_le 0.07021021
CTRY_2_le 0.01036221
P_NAME_le 0.012781565
storage_le 0.13574798
part_le 0.008187923
process_le 0.29719895
sashimi_le 0.32332537
steaming_processing_le 0.14218579


# TEST 

In [158]:
data = pd.read_excel('../input/bigcontest/2021 ___Biz__update_210831.xlsx')

In [159]:
make_test = pd.DataFrame(columns=['REG_DATE', 'CATEGORY_2', 'P_NAME', 'P_PRICE'])

In [160]:
make_test['REG_DATE'] = data['일자']
make_test['P_PRICE'] = 0
squid = deepcopy(make_test)
salmon = deepcopy(make_test)
shrimp = deepcopy(make_test)

In [161]:
squid['CATEGORY_2'] = '오징어'
squid['P_NAME'] = '오징어'
salmon['CATEGORY_2'] = '연어'
salmon['P_NAME'] = '연어'
shrimp['CATEGORY_2'] = '새우'
shrimp['P_NAME'] = '흰다리새우'

In [162]:
test = pd.concat([squid, salmon, shrimp]).reset_index(drop=True)
test_2 = test

In [163]:
test['YEAR'] = test['REG_DATE'].dt.year
test['MONTH'] = test['REG_DATE'].dt.month
test['DAY'] = test['REG_DATE'].dt.day

In [164]:
test['주차'] = None

In [165]:
for j in range(1,13):
    max_ = 32
    w = 1
    for i in test.index:
        if test.REG_DATE[i].month == j:
            x = test.REG_DATE[i].day
            if max_ >= x:
                max_ = x
                test['주차'][i] = w
            else:
                max_ = x
                w += 1
                test['주차'][i] = w

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


# CTRY_1 & CTRY_2 & P_PURPOSE

In [166]:
tmp = pd.DataFrame()
j = 0

for i in range(len(test)):
    cond1 = train['P_NAME']==test.loc[i, 'P_NAME']
    cond2 = train['MONTH']==test.loc[i, 'MONTH']
    cond3 = train['주차']==test.loc[i, '주차']
    cond = False
    for a, b, c in train.loc[cond1 & cond2 & cond3, ['CTRY_1', 'CTRY_2', 'P_PURPOSE']].value_counts().sort_values(ascending=False)[:3].index:
        cond = True
        tmp.loc[j, 'REG_DATE'] = test.loc[i, 'REG_DATE']
        tmp.loc[j, 'P_NAME'] = test.loc[i, 'P_NAME']
        tmp.loc[j, 'CTRY_1'] = a
        tmp.loc[j, 'CTRY_2'] = b
        tmp.loc[j, 'P_PURPOSE'] = c
        j += 1
    if not cond:
        for a, b, c in train.loc[cond1, ['CTRY_1', 'CTRY_2','P_PURPOSE']].value_counts()[:3].index:
            tmp.loc[j, 'REG_DATE'] = test.loc[i, 'REG_DATE']
            tmp.loc[j, 'P_NAME'] = test.loc[i, 'P_NAME']
            tmp.loc[j, 'CTRY_1'] = a
            tmp.loc[j, 'CTRY_2'] = b
            tmp.loc[j, 'P_PURPOSE'] = c
            j += 1

In [167]:
test = tmp.merge(test, how='outer', on=['REG_DATE', 'P_NAME'])

# STORAGE 예측

In [168]:
test = test.fillna('null')

In [169]:
str_FEATS = ['CTRY_1', 'CTRY_2', 'P_NAME', 'P_PURPOSE',]

In [170]:
for feat in str_FEATS:
    test[feat+'_le'] = test[feat].apply(lambda x: train.loc[train[feat]==x, feat+'_le'].unique()[0])

In [171]:
FEATS = ['주차', 'YEAR', 'MONTH','CTRY_1_le', 'CTRY_2_le',
       'P_NAME_le','P_PURPOSE_le']
TARGET = ['storage_le']

In [172]:
model = XGBClassifier(**params)

In [173]:
model.fit(train[FEATS],train[TARGET])

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


[04:15:36] WARNING: ../src/learner.cc:573: 
Parameters: { "devices", "iterations", "one_hot_max_size", "scale_pos_weight", "task_type" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[04:15:36] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, devices='0:1', gamma=0,
              gpu_id=-1, importance_type='gain', interaction_constraints='',
              iterations=500, learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=2, num_parallel_tree=1,
              objective='multi:softprob', one_hot_max_size=256, random_state=2,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              task_type='GPU', tree_method='exact', validate_parameters=1, ...)

In [174]:
predition = model.predict(test[FEATS])
test['storage_le'] = predition

# PART 예측

In [175]:
FEATS = [ 'YEAR', 'MONTH', '주차', 'CTRY_1_le', 'CTRY_2_le', 'P_NAME_le','storage_le', 'P_PURPOSE_le']
TARGET = ['part_le']

In [176]:
model = XGBClassifier(**params)

In [177]:
model.fit(train[FEATS],train[TARGET])

[04:15:44] WARNING: ../src/learner.cc:573: 
Parameters: { "devices", "iterations", "one_hot_max_size", "scale_pos_weight", "task_type" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[04:15:44] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, devices='0:1', gamma=0,
              gpu_id=-1, importance_type='gain', interaction_constraints='',
              iterations=500, learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=2, num_parallel_tree=1,
              objective='multi:softprob', one_hot_max_size=256, random_state=2,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              task_type='GPU', tree_method='exact', validate_parameters=1, ...)

In [178]:
predition = model.predict(test[FEATS])
test['part_le'] = predition

# PROCESS 예측

In [179]:
FEATS = [ 'YEAR', 'MONTH', '주차', 'CTRY_1_le', 'CTRY_2_le', 'P_NAME_le','storage_le','part_le', 'P_PURPOSE_le']
TARGET = ['process_le']

In [180]:
model = XGBClassifier(**params)

In [181]:
model.fit(train[FEATS],train[TARGET])

[04:16:09] WARNING: ../src/learner.cc:573: 
Parameters: { "devices", "iterations", "one_hot_max_size", "scale_pos_weight", "task_type" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[04:16:09] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, devices='0:1', gamma=0,
              gpu_id=-1, importance_type='gain', interaction_constraints='',
              iterations=500, learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=2, num_parallel_tree=1,
              objective='multi:softprob', one_hot_max_size=256, random_state=2,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              task_type='GPU', tree_method='exact', validate_parameters=1, ...)

In [182]:
predition = model.predict(test[FEATS])
test['process_le'] = predition

# SASHIMI예측

In [183]:
FEATS = [ 'YEAR', 'MONTH', '주차', 'CTRY_1_le', 'CTRY_2_le', 'P_NAME_le','storage_le','part_le', 'P_PURPOSE_le']
TARGET = ['sashimi_le']

In [184]:
model = XGBClassifier(**params)

In [185]:
model.fit(train[FEATS],train[TARGET])

[04:16:19] WARNING: ../src/learner.cc:573: 
Parameters: { "devices", "iterations", "one_hot_max_size", "scale_pos_weight", "task_type" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[04:16:19] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, devices='0:1', gamma=0,
              gpu_id=-1, importance_type='gain', interaction_constraints='',
              iterations=500, learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=2, num_parallel_tree=1,
              objective='multi:softprob', one_hot_max_size=256, random_state=2,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              task_type='GPU', tree_method='exact', validate_parameters=1, ...)

In [186]:
predition = model.predict(test[FEATS])
test['sashimi_le'] = predition

# STEAMING_PROCESSING 예측

In [188]:
params = {
    "iterations": 500,
    "learning_rate": 0.1,
    "task_type": "GPU",
    "devices": "0:1",
    'one_hot_max_size':256,
    'random_state':2
}

In [189]:
model_C = CatBoostClassifier(**params)

In [190]:
FEATS = [ 'YEAR', 'MONTH', '주차', 'CTRY_1_le', 'CTRY_2_le', 'P_NAME_le','storage_le','part_le','sashimi_le', 'P_PURPOSE_le']
TARGET = ['steaming_processing_le']

In [191]:
model_C.fit(train[FEATS],train[TARGET])

0:	learn: 0.4939278	total: 71.3ms	remaining: 35.6s
1:	learn: 0.3746446	total: 135ms	remaining: 33.6s
2:	learn: 0.3075324	total: 201ms	remaining: 33.2s
3:	learn: 0.2631550	total: 262ms	remaining: 32.5s
4:	learn: 0.2321036	total: 332ms	remaining: 32.9s
5:	learn: 0.2096915	total: 393ms	remaining: 32.4s
6:	learn: 0.1935464	total: 455ms	remaining: 32s
7:	learn: 0.1820895	total: 522ms	remaining: 32.1s
8:	learn: 0.1738029	total: 586ms	remaining: 32s
9:	learn: 0.1671024	total: 647ms	remaining: 31.7s
10:	learn: 0.1609670	total: 709ms	remaining: 31.5s
11:	learn: 0.1559244	total: 771ms	remaining: 31.3s
12:	learn: 0.1515219	total: 833ms	remaining: 31.2s
13:	learn: 0.1480274	total: 895ms	remaining: 31.1s
14:	learn: 0.1438441	total: 957ms	remaining: 30.9s
15:	learn: 0.1385941	total: 1.02s	remaining: 30.8s
16:	learn: 0.1358837	total: 1.08s	remaining: 30.7s
17:	learn: 0.1330082	total: 1.14s	remaining: 30.6s
18:	learn: 0.1291795	total: 1.2s	remaining: 30.5s
19:	learn: 0.1273272	total: 1.26s	remaining: 

In [192]:
predition = model_C.predict(test[FEATS])
test['steaming_processing_le'] = predition

# P_PRICE 예측

In [193]:
from xgboost import XGBRegressor

In [194]:
params = {
    "iterations": 500,
    "learning_rate": 0.1,
    "task_type": "GPU",
    "devices": "0:1",
    'one_hot_max_size':256,
    'objective':'reg:linear',
    'random_state':2
}


In [195]:
FEATS = ['CTRY_1_le','CTRY_2_le', 'P_NAME_le', 'storage_le', 'part_le',
       'process_le', 'sashimi_le', 'steaming_processing_le']
TARGETS = ['P_PRICE']

In [196]:
model_R = XGBRegressor(**params)

In [197]:
model_R.fit(train[FEATS],train[TARGETS])

[04:16:57] WARNING: ../src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[04:16:57] WARNING: ../src/learner.cc:573: 
Parameters: { "devices", "iterations", "one_hot_max_size", "task_type" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, devices='0:1', gamma=0,
             gpu_id=-1, importance_type='gain', interaction_constraints='',
             iterations=500, learning_rate=0.1, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=2, num_parallel_tree=1,
             objective='reg:linear', one_hot_max_size=256, random_state=2,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             task_type='GPU', tree_method='exact', validate_parameters=1, ...)

In [198]:
pred = model_R.predict(test[FEATS])
test["P_PRICE"] = pred

In [199]:
test.columns

Index(['REG_DATE', 'P_NAME', 'CTRY_1', 'CTRY_2', 'P_PURPOSE', 'CATEGORY_2',
       'P_PRICE', 'YEAR', 'MONTH', 'DAY', '주차', 'CTRY_1_le', 'CTRY_2_le',
       'P_NAME_le', 'P_PURPOSE_le', 'storage_le', 'part_le', 'process_le',
       'sashimi_le', 'steaming_processing_le'],
      dtype='object')

In [200]:
test

,REG_DATE,P_NAME,CTRY_1,CTRY_2,P_PURPOSE,CATEGORY_2,P_PRICE,YEAR,MONTH,DAY,주차,CTRY_1_le,CTRY_2_le,P_NAME_le,P_PURPOSE_le,storage_le,part_le,process_le,sashimi_le,steaming_processing_le
0,2021-01-04,오징어,중국,중국,판매용,오징어,2.316528,2021,1,4,1,65,64,183,3,1,9,0,0,0
1,2021-01-04,오징어,페루,페루,판매용,오징어,5.702712,2021,1,4,1,80,80,183,3,1,9,0,0,1
2,2021-01-04,오징어,대만,대만,판매용,오징어,3.524622,2021,1,4,1,12,11,183,3,1,0,0,0,0
3,2021-01-11,오징어,중국,중국,판매용,오징어,2.316528,2021,1,11,2,65,64,183,3,1,9,0,0,0
4,2021-01-11,오징어,페루,페루,판매용,오징어,5.702712,2021,1,11,2,80,80,183,3,1,9,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,2021-06-21,흰다리새우,태국,태국,판매용,새우,11.821100,2021,6,21,9,73,72,285,3,1,15,0,0,0
229,2021-06-21,흰다리새우,에콰도르,에콰도르,판매용,새우,6.983263,2021,6,21,9,54,53,285,3,1,0,0,0,0
230,2021-06-28,흰다리새우,베트남,베트남,판매용,새우,8.858169,2021,6,28,10,33,32,285,3,1,15,0,0,1
231,2021-06-28,흰다리새우,태국,태국,판매용,새우,11.821100,2021,6,28,10,73,72,285,3,1,15,0,0,0


In [201]:
pred1 = test.loc[test['P_NAME']=="오징어", :].groupby(["REG_DATE"])["P_PRICE"].mean().tolist()
pred2 = test.loc[test['P_NAME']=="연어", :].groupby(["REG_DATE"])["P_PRICE"].mean().tolist()
pred3 = test.loc[test['P_NAME']=="흰다리새우", :].groupby(["REG_DATE"])["P_PRICE"].mean().tolist()

prediction = pred1 + pred2 + pred3

In [202]:
test_2['P_PRICE'] = prediction

In [203]:
test_2.to_csv('test.csv',encoding='CP949')